**Claim extraction from text documents**

By fine tuning transformers

In [1]:
import pandas as pd

In [3]:
# load dataset
df = pd.read_csv('data/combined_dataset.csv')
"""
dataset consisting of
- claim
- y (0 -> noclaim 1 -> claim)
""" 
df.head()

,Unnamed: 0,claim,y
0,0,"At age 44, in 1972, Temple was diagnosed with ...",0
1,1,"Born in Vermont, son of a Methodist preacher w...",0
2,2,Ethiopia is an African country.,1
3,3,css pages in userspace can only be edited by t...,0
4,4,"Constitution states should, for the most part,...",0


In [8]:
# gather some info about data
class_counts = df['y'].value_counts()
print(f'the data consists of {len(df)} entries with\n - {class_counts[0]} being no claims\n - {class_counts[1]} being claims')

the data consists of 395057 entries with
 - 209612 being no claims
 - 185445 being claims


In [26]:
# load the dataset to huggingface class
from datasets import load_dataset

"""the data consists of 395057 entries with
 - 209612 being no claims
 - 185445 being claims
 """

# 70 / 30 split
train = load_dataset('csv', data_files='data/combined_dataset.csv', split='train[:70%]')
test = load_dataset('csv', data_files='data/combined_dataset.csv', split='train[70%:]')

print(train, test)

Using custom data configuration default-4a26f321fb9f2e54
Found cached dataset csv (/home/cas/.cache/huggingface/datasets/csv/default-4a26f321fb9f2e54/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration default-4a26f321fb9f2e54
Found cached dataset csv (/home/cas/.cache/huggingface/datasets/csv/default-4a26f321fb9f2e54/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Dataset({
    features: ['Unnamed: 0', 'claim', 'y'],
    num_rows: 276540
}) Dataset({
    features: ['Unnamed: 0', 'claim', 'y'],
    num_rows: 118517
})


In [27]:
# get tokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(batch):
    return tokenizer(batch["claim"], padding="max_length", truncation=True, return_tensors='pt')

In [23]:
# tokenize all train claims in advance
tokenized_train = train.map(tokenize_function, batched=True)

100%|██████████| 277/277 [00:31<00:00,  8.89ba/s]


In [34]:
tokenized_test = test.map(tokenize_function, batched=True)

100%|██████████| 119/119 [00:12<00:00,  9.42ba/s]


In [24]:
# load the distilbert to finetune from huggingface
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading: 100%|██████████| 256M/256M [00:08<00:00, 32.4MB/s] 


In [41]:
# load class for hyperparameters
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", 
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [33]:
# define evaluation metrics
import numpy as np
import evaluate

metric = evaluate.load("f1")

In [36]:
# copy pasta 🍝 (no idea what this does)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [42]:
from transformers import Trainer
# create the trainer to finetune with
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_test,
    eval_dataset=tokenized_train,
    compute_metrics=compute_metrics,
)

In [43]:
# start fine tuning! 🍿
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: Unnamed: 0, claim, y. If Unnamed: 0, claim, y are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/home/cas/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 118517
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 88890
  0%|          | 0/88890 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 3.94 GiB total capacity; 2.54 GiB already allocated; 20.56 MiB free; 2.57 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF